In [2]:
import os
import IPython

# Restart the kernel programmatically
IPython.Application.instance().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [1]:
!pip install \
  transformers==4.48.0 \
  accelerate==0.26.0 \
  peft==0.7.1 \
  datasets==2.14.5 \
  evaluate==0.4.1 \
  scikit-learn \
  wandb \
  ipywidgets \
  matplotlib \
  --upgrade -q


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import torch
import random
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, set_seed
from datasets import load_dataset
import wandb
import time

# Set seed for reproducibility
set_seed(42)
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Optional: Login to wandb (uncomment if not already done)
# wandb.login()

print("✅ Environment ready for LoRA training!")

✅ Environment ready for LoRA training!


In [2]:
# Load the labeled version of PubMedQA
dataset = load_dataset("pubmed_qa", "pqa_labeled")

# Create an 80/20 train-validation split
dataset = dataset["train"].train_test_split(test_size=0.2)
dataset["validation"] = dataset.pop("test")

# Reformat each example for BERT input
def format_for_bert(example):
    example["text"] = f"Question: {example['question']} Context: {example['context']}"
    label_map = {"yes": 0, "no": 1, "maybe": 2}
    example["label"] = label_map[example["final_decision"]]
    return example

dataset = dataset.map(format_for_bert)
print("✅ Dataset formatted for BERT classification")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

✅ Dataset formatted for BERT classification


In [3]:
from transformers import AutoTokenizer

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the input text
def tokenize_for_bert(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_token_type_ids=True  # Important for BERT
    )

# Apply to both train and validation sets
tokenized_dataset = dataset.map(
    tokenize_for_bert,
    batched=True,
    remove_columns=["pubid", "question", "context", "long_answer", "final_decision", "text"]
)

print("✅ Tokenization complete")

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

✅ Tokenization complete


In [4]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForSequenceClassification
import wandb
import time
import torch

# Initialize wandb with LoRA intent
if not wandb.run:
    wandb.init(
        project="peft-pubmedqa",
        name=f"bert-lora-{time.strftime('%Y%m%d-%H%M%S')}",
        tags=["bert", "lora", "peft"],
        config={
            "model": "bert-base-uncased",
            "method": "lora",  # Indicating the intended method
            "strategy": "peft",
            "lora_rank": 8,  # These are the intended LoRA parameters
            "lora_alpha": 16,
            "dataset": "pubmedqa",
            "seed": 42,
            "note": "Implementation uses standard fine-tuning due to library compatibility issues"
        }
    )

# Load the base BERT model for classification
print("🔄 Loading base model...")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],  # works well with BERT attention layers
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Wrap the model with LoRA
print("➕ Applying LoRA...")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"🚀 Model moved to {device}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: chandantroughia (chandantroughia-cst) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


🔄 Loading base model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


➕ Applying LoRA...
trainable params: 297,219 || all params: 109,781,766 || trainable%: 0.27073621679578375
🚀 Model moved to cuda


In [12]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import os

# Create output directory
output_dir = "./results/bert-lora-finetune"
os.makedirs(output_dir, exist_ok=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    fp16=torch.cuda.is_available(),
    report_to="wandb",
    logging_steps=50,
    push_to_hub=False,
    seed=42
)

# Load evaluation metrics
accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")

# Metric computation function
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="macro")  # For multiclass
    
    return {
        "accuracy": acc,
        "f1": f1
    }

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,  # your model should already be moved to GPU (if available)
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
)

# Start training
print("⏱️ Starting training...")
trainer.train()

⏱️ Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.942700,0.975630,0.515000,0.226623
2,0.951400,0.982561,0.515000,0.226623
3,0.991000,0.989175,0.515000,0.226623


TrainOutput(global_step=300, training_loss=0.9354345703125, metrics={'train_runtime': 73.0473, 'train_samples_per_second': 32.855, 'train_steps_per_second': 4.107, 'total_flos': 633663538790400.0, 'train_loss': 0.9354345703125, 'epoch': 3.0})

In [17]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import numpy as np

# ✅ Step 1: Evaluate using built-in evaluate()
print("📊 Evaluating the LoRA model on validation set...")
eval_results = trainer.evaluate()
print(f"\n✅ Evaluation Results: {eval_results}")

# ✅ Step 2: Manual Predictions
print("\n🔍 Running manual predictions...")
model.eval()
all_preds = []
all_labels = []

# Process validation dataset in batches
batch_size = 16
num_examples = len(tokenized_dataset["validation"])

with torch.no_grad():
    for i in range(0, num_examples, batch_size):
        end_idx = min(i + batch_size, num_examples)
        batch_data = tokenized_dataset["validation"][i:end_idx]
        
        # Convert to appropriate format
        input_ids = torch.tensor(batch_data["input_ids"]).to(device)
        attention_mask = torch.tensor(batch_data["attention_mask"]).to(device)
        token_type_ids = torch.tensor(batch_data["token_type_ids"]).to(device)
        labels = torch.tensor(batch_data["label"]).to(device)  # Note: using "label" (singular)
        
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        # Get predictions
        predictions = torch.argmax(outputs.logits, dim=-1)
        
        # Store results
        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ✅ Step 3: Decode numeric predictions to labels
label_map_reverse = {0: "yes", 1: "no", 2: "maybe"}
pred_texts = [label_map_reverse[p] for p in all_preds]
label_texts = [label_map_reverse[l] for l in all_labels]

# ✅ Step 4: Metrics
accuracy = sum(p == l for p, l in zip(all_preds, all_labels)) / len(all_preds)
print(f"\n📈 Manual Accuracy: {accuracy:.4f}")

print("\n📋 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["yes", "no", "maybe"]))

print("\n🔎 Sample Predictions:")
for i in range(min(10, len(pred_texts))):
    match = "✓" if pred_texts[i] == label_texts[i] else "✗"
    print(f"Example {i+1}: Prediction: {pred_texts[i]}, True label: {label_texts[i]} {match}")

📊 Evaluating the LoRA model on validation set...

✅ Evaluation Results: {'eval_loss': 0.9891747832298279, 'eval_accuracy': 0.515, 'eval_f1': 0.22662266226622663, 'eval_runtime': 1.0456, 'eval_samples_per_second': 191.279, 'eval_steps_per_second': 23.91, 'epoch': 3.0}

🔍 Running manual predictions...

📈 Manual Accuracy: 0.5150

📋 Classification Report:
              precision    recall  f1-score   support

         yes       0.52      1.00      0.68       103
          no       0.00      0.00      0.00        72
       maybe       0.00      0.00      0.00        25

    accuracy                           0.52       200
   macro avg       0.17      0.33      0.23       200
weighted avg       0.27      0.52      0.35       200


🔎 Sample Predictions:
Example 1: Prediction: yes, True label: yes ✓
Example 2: Prediction: yes, True label: no ✗
Example 3: Prediction: yes, True label: yes ✓
Example 4: Prediction: yes, True label: maybe ✗
Example 5: Prediction: yes, True label: maybe ✗
Example 6

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
